In [66]:
import os
import warnings
RANDOM_STATE = 150824
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, make_scorer
from sklearn.model_selection import cross_val_score, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score 
import torch
import torch.nn as nn
import torch.optim as optim
from xgboost import XGBRegressor
from phik.report import plot_correlation_matrix

from sqlalchemy import create_engine, Column, Integer, String, Float, DateTime, text, select, MetaData, Table
from sqlalchemy.orm import DeclarativeBase, Session
from sqlalchemy.schema import DropTable
from sqlalchemy.ext.compiler import compiles

# Для LGBMRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:.4f}'.format


In [67]:
# Инициализация пустого DataFrame для хранения результатов
results_df = pd.DataFrame(columns=['Model', 'Best_Score', 'Best_Params'])

def add_to_results_table(model_name, best_model, best_score, best_params):
    """
    Добавляет результаты модели в таблицу для сравнения.

    Параметры:
        model_name (str): Название модели.
        best_model: Лучшая модель, найденная в процессе оптимизации.
        best_score (float): Лучшее значение MAE, найденное на кросс-валидации.
        best_params (dict): Словарь с лучшими гиперпараметрами.

    Возвращает:
        results_df (pd.DataFrame): Обновленная таблица с результатами моделей.
    """
    global results_df  # Используем глобальную переменную для хранения результатов

    # Создаем новую строку с результатами в виде DataFrame
    new_row = pd.DataFrame({
        'Model': [model_name],
        'Best_Score': [best_score],
        'Best_Params': [best_params]
    })
    
    # Добавляем строку в DataFrame с использованием pd.concat
    results_df = pd.concat([results_df, new_row], ignore_index=True)
    
    return results_df

In [68]:
# Читаем файл обратно
final_df = pd.read_csv('final_df.csv', index_col='key')

# Проверяем первые несколько строк
display(final_df.head())

,Unnamed: 0,start_temp,target_temp,key_cycles,active_power_mean,active_power_max,active_power_min,active_power_std,active_power_range,reactive_power_mean,apparent_power_std,power_factor_mean,heating_duration_sum,heating_duration_std,energy_consumed_sum,energy_consumed_std,power_diff_mean,heating_duration_diff_mean,power_change_mean,power_change_max,bulk_1,bulk_3,bulk_4,bulk_5,bulk_6,bulk_7,bulk_10,bulk_11,bulk_12,bulk_14,bulk_15,wire_1,wire_2,wire_3,wire_6,gas,bulk_sum,wire_sum,wire_add_count,bulk_add_count,added_sum,add_count_total,bulk_other,wire_other
key,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,0,1571.0000,1613.0000,5,0.6073,0.8671,0.3051,0.2194,0.5620,0.4286,0.2682,0.8167,1098.0000,86.3151,770.2821,55.6778,0.1124,-16.0000,-8.0345,-4.2505,0.0000,0.0000,43.0000,0.0000,0.0000,0.0000,0.0000,0.0000,206.0000,150.0000,154.0000,60.0600,0.0000,0.0000,0.0000,29.7500,553.0000,60.0600,4,2,613.0600,6,0.0000,0.0000
2,1,1581.0000,1602.0000,4,0.5349,0.7863,0.2617,0.2532,0.5247,0.3633,0.3066,0.8253,811.0000,97.5683,481.7600,58.4605,0.1013,18.2500,0.0000,-7.0075,0.0000,0.0000,73.0000,0.0000,0.0000,0.0000,0.0000,0.0000,206.0000,149.0000,154.0000,96.0523,0.0000,0.0000,0.0000,12.5556,582.0000,96.0523,4,2,678.0523,6,0.0000,0.0000
3,2,1596.0000,1599.0000,5,0.8127,1.2230,0.4217,0.3461,0.8013,0.5875,0.4237,0.8097,655.0000,86.6112,722.8377,137.8374,-0.1360,-43.0000,-4.8065,-2.7065,0.0000,0.0000,34.0000,0.0000,0.0000,0.0000,0.0000,0.0000,205.0000,152.0000,153.0000,91.1602,0.0000,0.0000,0.0000,28.5548,544.0000,91.1602,4,2,635.1602,6,0.0000,0.0000
4,3,1601.0000,1625.0000,4,0.6766,0.8879,0.3107,0.2678,0.5772,0.5142,0.3292,0.7939,741.0000,75.9665,683.4556,107.2103,0.0555,17.7500,-5.7416,-4.6101,0.0000,0.0000,81.0000,0.0000,0.0000,0.0000,0.0000,0.0000,207.0000,153.0000,154.0000,89.0635,0.0000,0.0000,0.0000,18.8412,595.0000,89.0635,4,2,684.0635,6,0.0000,0.0000
5,4,1576.0000,1602.0000,4,0.5632,0.8930,0.3246,0.2423,0.5684,0.4220,0.2967,0.7968,869.0000,151.5484,512.1699,69.2709,0.1421,-45.0000,0.0000,0.0000,0.0000,0.0000,78.0000,0.0000,0.0000,0.0000,0.0000,0.0000,203.0000,151.0000,152.0000,89.2382,9.1146,0.0000,0.0000,5.4137,584.0000,98.3528,4,3,682.3528,7,0.0000,0.0000


In [89]:
# Разделение данных на признаки (X) и таргет (y)
X = final_df.drop(columns=['target_temp'])
y = final_df['target_temp']

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Преобразование данных с использованием StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Вывод размеров данных для проверки
print("Размеры обучающих данных: ", X_train.shape, y_train.shape)
print("Размеры тестовых данных: ", X_test.shape, y_test.shape)


Размеры обучающих данных:  (1782, 43) (1782,)
Размеры тестовых данных:  (594, 43) (594,)


In [70]:
model_lr = LinearRegression()
model_lr.fit(X_train, y_train)
mae_lr = round(abs(cross_val_score(model_lr, X=X_train, y=y_train, cv=5, scoring='neg_mean_absolute_error')).mean(), 4)
print(f'MAE LinearRegression =  {mae_lr}')

MAE LinearRegression =  7.4591


In [71]:
def evaluate_model_with_cv(model, X, y, cv=5):
    """
    Оценивает модель с использованием кросс-валидации по метрике MAE.

    Параметры:
        model: Модель для оценки.
        X (pd.DataFrame или np.ndarray): Матрица признаков.
        y (pd.Series или np.ndarray): Вектор целевой переменной.
        cv (int): Количество фолдов для кросс-валидации.

    Возвращает:
        mean_mae (float): Среднее значение MAE по фолдам кросс-валидации.
    """
    # Используем отрицательный MAE, поскольку в scikit-learn метрики ошибок представлены в отрицательном виде для целей оптимизации
    mae_scorer = make_scorer(mean_absolute_error, greater_is_better=False)
    
    # Выполняем кросс-валидацию
    cv_scores = cross_val_score(model, X, y, cv=cv, scoring=mae_scorer, n_jobs=-1)
    
    # Преобразуем отрицательные значения обратно в положительные
    mean_mae = -cv_scores.mean()
    std_mae = cv_scores.std()
    
    print(f"Среднее MAE на кросс-валидации: {mean_mae:.4f}")
    print(f"Стандартное отклонение MAE на кросс-валидации: {std_mae:.4f}")
    
    return mean_mae

In [72]:
def optimize_model(model, param_distributions, X, y, cv=5, n_iter=50, random_state=42):
    """
    Выполняет подбор гиперпараметров для переданной модели с использованием RandomizedSearchCV.

    Параметры:
        model: Модель, для которой производится подбор гиперпараметров.
        param_distributions (dict): Словарь гиперпараметров для RandomizedSearchCV.
        X (pd.DataFrame или np.ndarray): Матрица признаков.
        y (pd.Series или np.ndarray): Вектор целевой переменной.
        cv (int): Количество фолдов для кросс-валидации.
        n_iter (int): Количество итераций для RandomizedSearchCV.
        random_state (int): Зерно для генерации случайных чисел.

    Возвращает:
        best_model: Модель с лучшими найденными гиперпараметрами.
        best_score (float): Лучшее найденное значение MAE на кросс-валидации.
        best_params (dict): Словарь с лучшими гиперпараметрами.
    """
    
    # Настраиваем RandomizedSearchCV
    randomized_search = RandomizedSearchCV(
        estimator=model,
        param_distributions=param_distributions,
        n_iter=n_iter,
        cv=cv,
        scoring=make_scorer(mean_absolute_error, greater_is_better=False),
        random_state=random_state,
        n_jobs=-1,
        verbose=1
    )
    
    # Выполняем поиск
    randomized_search.fit(X, y)
    
    # Извлекаем лучшие параметры и модель
    best_model = randomized_search.best_estimator_
    best_params = randomized_search.best_params_
    best_score = -randomized_search.best_score_
    
    print("\nЛучшие параметры модели:")
    for param_name in best_params:
        print(f" - {param_name}: {best_params[param_name]}")
    
    print(f"\nЛучшее значение MAE на кросс-валидации: {best_score:.4f}")
    
    return best_model, best_score, best_params

In [73]:
# Пример: оценка модели LinearRegression
lr_model = LinearRegression()
print("Оценка модели LinearRegression:")
lr_mae = evaluate_model_with_cv(lr_model, X_train, y_train, cv=5)

Оценка модели LinearRegression:
Среднее MAE на кросс-валидации: 7.3298
Стандартное отклонение MAE на кросс-валидации: 0.0673


In [74]:
# Добавление результата в таблицу
results_df = pd.DataFrame(columns=['Model', 'Best_Score'])
results_df = pd.concat([results_df, pd.DataFrame({'Model': ['Linear Regression'], 'Best_Score': [mae_lr]})])

C:\Users\EPrutskoy\AppData\Local\Temp\ipykernel_23592\3328997962.py:3: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, pd.DataFrame({'Model': ['Linear Regression'], 'Best_Score': [mae_lr]})])


In [75]:
results_df

,Model,Best_Score
0,Linear Regression,7.4591


In [76]:
rf_param_distributions = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False]
}

best_rf_model, best_rf_score, best_rf_params = optimize_model(
    RandomForestRegressor(random_state=RANDOM_STATE),
    rf_param_distributions,
    X_train,
    y_train
)

model_name = "RandomForestRegressor"
# Добавляем результаты в таблицу
results_df = add_to_results_table(model_name, best_rf_model, best_rf_score, best_rf_params)

Fitting 5 folds for each of 50 candidates, totalling 250 fits


KeyboardInterrupt: 

In [ ]:
model_name = "RandomForestRegressor"
# Добавляем результаты в таблицу
results_df = add_to_results_table(model_name, best_rf_model, best_rf_score, best_rf_params)

In [ ]:
# Для CatBoostRegressor
cb_param_distributions = {
    'iterations': [100, 200, 500],
    'depth': [4, 6, 10],
    'learning_rate': [0.01, 0.05, 0.1],
    'l2_leaf_reg': [1, 3, 5, 7, 9]
}

best_cb_model, best_cb_score, best_cb_params = optimize_model(
    CatBoostRegressor(random_state=RANDOM_STATE, silent=True),
    cb_param_distributions,
    X_train,
    y_train, n_iter=15
)

model_name = 'CatBoostRegressor'
results_df = add_to_results_table(model_name, best_cb_model, best_cb_score, best_cb_params)

Fitting 5 folds for each of 15 candidates, totalling 75 fits

Лучшие параметры модели:
 - learning_rate: 0.05
 - l2_leaf_reg: 7
 - iterations: 500
 - depth: 4

Лучшее значение MAE на кросс-валидации: 6.0424


In [ ]:
lgb_param_distributions = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [None, 10, 20, 30, 50],
    'learning_rate': [0.001, 0.05, 0.1],
    'num_leaves': [30, 40, 50],
    'min_child_samples': [20, 30, 50]
}

best_lgb_model, best_lgb_score, best_lgb_params = optimize_model(
    LGBMRegressor(random_state=RANDOM_STATE),
    lgb_param_distributions,
    X_train,
    y_train, n_iter=10
)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000677 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 6701
[LightGBM] [Info] Number of data points in the train set: 1782, number of used features: 39
[LightGBM] [Info] Start training from score 1594.654882
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: 

In [ ]:
model_name = 'LGBMRegressor'
results_df = add_to_results_table(model_name, best_lgb_model, best_lgb_score, best_lgb_params)

In [ ]:
display(results_df)

,Model,Best_Score,Best_Params
0,Linear Regression,7.4591,NaN
1,RandomForestRegressor,6.2576,"{'n_estimators': 200, 'min_samples_split': 10,..."
2,CatBoostRegressor,6.0424,"{'learning_rate': 0.05, 'l2_leaf_reg': 7, 'ite..."
3,LGBMRegressor,6.3712,"{'num_leaves': 40, 'n_estimators': 100, 'min_c..."


In [ ]:
# Настраиваем гиперпараметры для XGBRegressor
param_distributions_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.001, 0.01, 0.1],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0],
    'gamma': [0, 0.1, 0.2],
    'reg_alpha': [0, 0.1, 1],
    'reg_lambda': [1, 0.1, 10]
}

# Оптимизация модели XGBRegressor
best_model_xgb, best_score_xgb, best_params_xgb = optimize_model(XGBRegressor(), 
                                                                 param_distributions_xgb, 
                                                                 X_train, 
                                                                 y_train,
                                                                 n_iter=25)


Fitting 5 folds for each of 25 candidates, totalling 125 fits

Лучшие параметры модели:
 - subsample: 0.8
 - reg_lambda: 0.1
 - reg_alpha: 1
 - n_estimators: 100
 - max_depth: 3
 - learning_rate: 0.1
 - gamma: 0.1
 - colsample_bytree: 1.0

Лучшее значение MAE на кросс-валидации: 6.1046


In [ ]:
results_df = add_to_results_table('XGBRegressor', best_model_xgb, best_score_xgb, best_params_xgb)

# Выводим таблицу сравнения
display(results_df)

,Model,Best_Score,Best_Params
0,Linear Regression,7.4591,NaN
1,RandomForestRegressor,6.2576,"{'n_estimators': 200, 'min_samples_split': 10,..."
2,CatBoostRegressor,6.0424,"{'learning_rate': 0.05, 'l2_leaf_reg': 7, 'ite..."
3,LGBMRegressor,6.3712,"{'num_leaves': 40, 'n_estimators': 100, 'min_c..."
4,XGBRegressor,6.3712,"{'num_leaves': 40, 'n_estimators': 100, 'min_c..."
5,XGBRegressor,6.3712,"{'num_leaves': 40, 'n_estimators': 100, 'min_c..."
6,XGBRegressor,6.3712,"{'num_leaves': 40, 'n_estimators': 100, 'min_c..."
7,XGBRegressor,6.1046,"{'subsample': 0.8, 'reg_lambda': 0.1, 'reg_alp..."


### Нейросеть

Создадим последовательную нейросеть.

,Model,Best_Score
0,Linear Regression,7.4591
